<a href="https://colab.research.google.com/github/Samenergy/Databases_P13/blob/main/laptop_price_prediction_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import joblib  # To load the trained model
from sklearn.preprocessing import StandardScaler

In [ ]:
def fetch_latest_laptop():
    """Fetches the latest laptop data from an API."""
    api_url = "https://api.example.com/latest-laptop"  # Replace with actual API URL
    response = requests.get(api_url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Failed to fetch data from API")

In [ ]:
def preprocess_laptop_data(raw_data, scaler):
    """Prepares the latest laptop data for price prediction."""
    df = pd.DataFrame([raw_data])

    # Feature Engineering
    processor_mapping = {'Intel i3': 1, 'Intel i5': 2, 'Intel i7': 3, 'Intel i9': 4,
                         'AMD Ryzen 3': 1, 'AMD Ryzen 5': 2, 'AMD Ryzen 7': 3, 'AMD Ryzen 9': 4}
    df['Processor'] = df['Processor'].map(processor_mapping)

    df['Storage (GB)'] = df['Storage'].apply(lambda x: int(x.replace('GB SSD', '').replace('TB SSD', '')) * (1000 if 'TB' in x else 1))

    gpu_mapping = {'Integrated': 0, 'Nvidia GTX 1650': 1, 'Nvidia RTX 2060': 2, 'Nvidia RTX 3060': 2, 'Nvidia RTX 3080': 3,
                   'AMD Radeon RX 6600': 2, 'AMD Radeon RX 6800': 3}
    df['GPU'] = df['GPU'].map(gpu_mapping)

    df['Resolution'] = df['Resolution'].apply(lambda res: int(res.split('x')[0]) * int(res.split('x')[1]))

    os_mapping = {'Windows': 3, 'macOS': 2, 'Linux': 1, 'FreeDOS': 0}
    df['Operating System'] = df['Operating System'].map(os_mapping)

    # One-hot encode 'Brand' (assuming the model was trained with one-hot encoding)
    brands = ['Apple', 'Asus', 'Dell', 'HP', 'Lenovo', 'MSI', 'Microsoft', 'Razer', 'Samsung']
    for brand in brands:
        df[f'Brand_{brand}'] = 1 if raw_data['Brand'] == brand else 0

    # Standardize numeric features
    df.iloc[:, :7] = scaler.transform(df.iloc[:, :7])

    return df

In [ ]:
def predict_laptop_price():
    """Fetch latest laptop data, preprocess it, and predict the price."""
    raw_data = fetch_latest_laptop()

    # Load trained model & scaler
    model = joblib.load("laptop_price_model.pkl")  # Replace with actual model path
    scaler = joblib.load("scaler.pkl")  # Replace with actual scaler path

    processed_data = preprocess_laptop_data(raw_data, scaler)

    # Predict price
    predicted_price = model.predict(processed_data)[0]

    print(f"Predicted Laptop Price: ${predicted_price:.2f}")

# Run prediction script
if __name__ == "__main__":
    predict_laptop_price()
